# Imports

In [ ]:
# Imports
import pandas as pd

import requests
import yaml
import json
import os
import firebase_admin
from datetime import datetime

#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

from dotenv import load_dotenv
 
load_dotenv(dotenv_path='/Users/daniel/OAN/credentials/contoan/.env')

shop_url = os.environ['SHOPIFY_ACCESS_URL']


In [ ]:
pd.set_option('max_colwidth', 2000)

## ACCESS FIREBASE

In [ ]:
filename=os.environ['FIREBASE_FILENAME']

In [ ]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## GET DATA FROM FIREBASE

In [ ]:
cuentas = db.collection('Accounting').where(u'execution_date',u'<',u'2021-00-00'
                                           ).where('type','==','expense'
                                               ).where('place','==','spain'
                                                   ).stream()

In [ ]:
l_cuentas = []
for cuenta in cuentas:
    #print(payout)
    l_cuentas.append(cuenta.to_dict())

In [ ]:
date_min = (pd.to_datetime(pd.DataFrame(l_payouts)['execution_date'].max()) + pd.DateOffset(1)).isoformat()[0:10]
date_min

### GET DATA FROM BIGQUERY

In [ ]:
df_cuentas = pd.read_gbq('''SELECT acc.id as `ID`,
                               acc.creation_date as `CREACION`,
                               acc.execution_date as `EJECUCION`,
                               acc.concept as `CONCEPTO`,
                               pr.name as `PROYECTO`,
                               int.name as `INTERVENCION`,
                               ph.name as `FASE`,
                               CASE 
                                WHEN acc.type = 'expense'  THEN 'Gasto'
                                WHEN acc.type = 'income'  THEN 'Ingreso'
                                WHEN acc.type = 'movement'  THEN 'Movimiento'
                                ELSE ''
                              END AS `TIPO`,
                               acc.amount as `CANTIDAD`,
                               acc.vat as `IVA`,
                               acc.vat_amount as `IVA_CTA`,
                               tg.name as `CAJA_BANCO`,
                               ac.name as `CATEGORIA`,
                               acc.description as `DESCRIPCION`,
                               cr.name as `CREADOR`,
                               rs.name as `RESPONSABLE`,      
                               acc.code as `COTADOR`,
                               acc.origin as `ORIGEN`,
                               acc.place as `LUGAR`,
                               acc.receipt as `FACTURA`


                        FROM `oan-nikarit.contoan.accounting` acc
                        LEFT OUTER JOIN `oan-nikarit.contoan.accounts` ac
                        ON acc.account_id = ac.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.accounts` tg
                        ON acc.target_id = tg.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.projects` pr
                        ON acc.project = pr.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.projects` int
                        ON acc.intervention = int.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.projects` ph
                        ON acc.phase = ph.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.users` cr
                        ON acc.creator_user = cr.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.users` rs
                        ON acc.user_in_charge = rs.id
                        
                        WHERE acc.place = 'spain'
                        AND acc.type = 'expense'
                        AND TIMESTAMP('2020-01-01') <= acc.execution_date
                        AND acc.origin = 'contoan'
                        '''
           )

In [ ]:
df_cuentas = df_cuentas.sort_values(by='EJECUCION').reset_index(drop=True)

In [ ]:
df_cuentas.shape

In [ ]:
df_cuentas['URL'] = 'https://nikarit.oaninternational.org/accounting/details/'+df_cuentas['ID']

In [ ]:
df_cuentas

In [ ]:
df_cuentas[df_cuentas['FACTURA'].isna()][['URL','EJECUCION','CANTIDAD','CONCEPTO','FACTURA']]

### SEUR

In [ ]:
filt = df_cuentas['CONCEPTO'].apply(lambda x: 'seur' in x.lower())
df_cuentas[filt][['URL','EJECUCION','CANTIDAD','CONCEPTO','FACTURA']]

### FB

In [ ]:
filt = df_cuentas['CONCEPTO'].apply(lambda x: 'facebook' in x.lower())
df_cuentas[filt][['URL','EJECUCION','CANTIDAD','CONCEPTO','FACTURA','CAJA_BANCO']]

### Google ADS

In [ ]:
filt = df_cuentas['CONCEPTO'].apply(lambda x: 'google' in x.lower())
df_cuentas[filt][['URL','EJECUCION','CANTIDAD','CONCEPTO','FACTURA','CAJA_BANCO']]

### Paypal

In [ ]:
filt = df_cuentas['CAJA_BANCO'].apply(lambda x: 'paypal' in x.lower())
df_cuentas[filt][['URL','EJECUCION','CANTIDAD','CONCEPTO','FACTURA','CAJA_BANCO']]

In [ ]:
df_cuentas = pd.read_gbq('''SELECT acc.id as `ID`,
                               acc.creation_date as `CREACION`,
                               acc.execution_date as `EJECUCION`,
                               acc.concept as `CONCEPTO`,
                               pr.name as `PROYECTO`,
                               int.name as `INTERVENCION`,
                               ph.name as `FASE`,
                               CASE 
                                WHEN acc.type = 'expense'  THEN 'Gasto'
                                WHEN acc.type = 'income'  THEN 'Ingreso'
                                WHEN acc.type = 'movement'  THEN 'Movimiento'
                                ELSE ''
                              END AS `TIPO`,
                               acc.amount as `CANTIDAD`,
                               acc.vat as `IVA`,
                               acc.vat_amount as `IVA_CTA`,
                               tg.name as `CAJA_BANCO`,
                               ac.name as `CATEGORIA`,
                               acc.description as `DESCRIPCION`,
                               cr.name as `CREADOR`,
                               rs.name as `RESPONSABLE`,      
                               acc.code as `COTADOR`,
                               acc.origin as `ORIGEN`,
                               acc.place as `LUGAR`,
                               acc.receipt as `FACTURA`


                        FROM `oan-nikarit.contoan.accounting` acc
                        LEFT OUTER JOIN `oan-nikarit.contoan.accounts` ac
                        ON acc.account_id = ac.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.accounts` tg
                        ON acc.target_id = tg.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.projects` pr
                        ON acc.project = pr.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.projects` int
                        ON acc.intervention = int.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.projects` ph
                        ON acc.phase = ph.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.users` cr
                        ON acc.creator_user = cr.id
                        LEFT OUTER JOIN `oan-nikarit.contoan.users` rs
                        ON acc.user_in_charge = rs.id
                        
                        WHERE acc.place = 'spain'
                        AND TIMESTAMP('2020-12-01') <= acc.execution_date
                        AND acc.origin = 'contoan'
                        ORDER BY acc.execution_date
                        
                        '''
           )

In [ ]:
df_cuentas['URL'] = 'https://nikarit.oaninternational.org/accounting/details/'+df_cuentas['ID']

In [ ]:
df_cuentas[df_cuentas['CAJA_BANCO'].apply(lambda x: 'ingenieros' in x.lower())
          ][['URL','EJECUCION','CANTIDAD','TIPO','CONCEPTO','FACTURA','CAJA_BANCO']
           ][0:30]

In [ ]:
df_cuentas[df_cuentas['CAJA_BANCO'].apply(lambda x: 'ingenieros' in x.lower())
          ][['URL','EJECUCION','CANTIDAD','TIPO','CONCEPTO','FACTURA','CAJA_BANCO']
           ][30:]

In [ ]:
df_cuentas[df_cuentas['CATEGORIA'].apply(lambda x: 'ingenieros' in x.lower())
          ][['URL','EJECUCION','CANTIDAD','TIPO','CONCEPTO','FACTURA','CAJA_BANCO']
           ][0:30]